this is the first primer of sorts to understanding a transformer. we build a bigram model which takes in a shakespeare dataset, and trains to predict the next token, but considering just 1 past token.

this would give us a good intuition for the future transformer model we will build.

> Add blockquote



In [1]:
# We will begin by download the tiny shakespeare dataset, which consists of all of his works in a single txt file
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-03-02 04:17:51--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2025-03-02 04:17:51 (25.1 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [3]:
print("Size of the dataset (num of characters):", len(text))

Size of the dataset (num of characters): 1115394


In [4]:
#Exploring the vocabulary of our dataset to see the different characters present in it
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'Size if vocab is: {vocab_size}\n and the characters are: {"".join(chars)}')

Size if vocab is: 65
 and the characters are: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [5]:
#Building our simple Tokenizer that simply converts our vocab to a unique integer
ch_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_ch = {i:ch for i,ch in enumerate(chars)}

encoder = lambda str: [ch_to_int[char] for char in str] #encodes each character in a string with it's unique int val
decoder = lambda int: ''.join(int_to_ch[i] for i in int) #decodes the characters in their integer forms

In [6]:
#testing out our tokenizer
print(encoder('penguins are the best'))
print(decoder(encoder('penguins are the best')))

[54, 43, 52, 45, 59, 47, 52, 57, 1, 39, 56, 43, 1, 58, 46, 43, 1, 40, 43, 57, 58]
penguins are the best


In [7]:
#applying our tokenizer to our dataset using pytorch

import torch
data = torch.tensor(encoder(text), dtype=torch.long) #encode and convert to torch type tensor
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [8]:
#let's look at a small part of our dataset, before and after encoding
print('Before:')
print(text[:1000])

Before:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for re

In [9]:
print('After:')
print(data[:1000])

After:
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  

In [10]:
# getting our dataset ready for training

#first we split our dataset into train and test sets.
# 80% will be kept for train and the rest for test

n = int(0.8*len(data))
train_data = data[:n]
test_data = data[n:]



In [11]:
#chunking or splitting up the train set into chunks/blocks of size 9

# we chunk them up so that the model can learn the importance of 9 characters and their relationship with each other, such that given the first char it will be able to predict the next token (char).
#this is the max context size/length for predictions
chunk_size = 9

#example
print(train_data[:chunk_size+1])
print(decoder((train_data[:chunk_size+1]).numpy()))

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])
First Citi


In [12]:
#let's demonstrate how the above would work

x = train_data[:chunk_size]
y = train_data[1:chunk_size+1]

for t in range(chunk_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) the target: 47


In [13]:
#let's split our training data into batches for efficient parallelism

torch.manual_seed(22)
batch_size = 4 #split our training to 4 parallel batches
chunk_size = 9

def fetch_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - chunk_size, (batch_size,)) #returns batch_size number of random integers between 0 and len(data) - chunksize
  #These random integers will be used as starting indices for selecting chunks of data.
  #now that we have a batch of random parts of our data, we classify them into x (dependent vars) and y (target)
  x = torch.stack([data[i:i+chunk_size] for i in ix])
  #here x is a stack or multiple rows (batch size num of rows) to act as input data
  y = torch.stack([data[i+1:i+chunk_size+1] for i in ix])
  #here y is a stack or multiple rows to act as target/output data for each tensor rows in input

  return x,y

In [14]:
xb, yb = fetch_batch('train')
print('The inputs are:')
print(xb.shape)
print(xb)

print('The ouputs are:')
print(yb.shape)
print(yb)

#As you can see below the input and outputs will be of shape [batch_size, chunk_size]

The inputs are:
torch.Size([4, 9])
tensor([[44,  8,  0, 32, 46, 47, 57,  1, 56],
        [50, 47, 60, 43,  0, 32, 53,  1, 40],
        [61, 52,  6,  0, 31, 47, 52, 41, 43],
        [ 1, 40, 43,  1, 53, 52, 43,  8,  0]])
The ouputs are:
torch.Size([4, 9])
tensor([[ 8,  0, 32, 46, 47, 57,  1, 56, 53],
        [47, 60, 43,  0, 32, 53,  1, 40, 43],
        [52,  6,  0, 31, 47, 52, 41, 43,  1],
        [40, 43,  1, 53, 52, 43,  8,  0, 14]])


In [15]:
#Let's demonstrate a similar prediction example like before:

for b in range(batch_size):
  for t in range(chunk_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target: {target}")

when input is [44] the target: 8
when input is [44, 8] the target: 0
when input is [44, 8, 0] the target: 32
when input is [44, 8, 0, 32] the target: 46
when input is [44, 8, 0, 32, 46] the target: 47
when input is [44, 8, 0, 32, 46, 47] the target: 57
when input is [44, 8, 0, 32, 46, 47, 57] the target: 1
when input is [44, 8, 0, 32, 46, 47, 57, 1] the target: 56
when input is [44, 8, 0, 32, 46, 47, 57, 1, 56] the target: 53
when input is [50] the target: 47
when input is [50, 47] the target: 60
when input is [50, 47, 60] the target: 43
when input is [50, 47, 60, 43] the target: 0
when input is [50, 47, 60, 43, 0] the target: 32
when input is [50, 47, 60, 43, 0, 32] the target: 53
when input is [50, 47, 60, 43, 0, 32, 53] the target: 1
when input is [50, 47, 60, 43, 0, 32, 53, 1] the target: 40
when input is [50, 47, 60, 43, 0, 32, 53, 1, 40] the target: 43
when input is [61] the target: 52
when input is [61, 52] the target: 6
when input is [61, 52, 6] the target: 0
when input is [61,

In [16]:
#we mainly capture x(input) for the model to train on and the target y (output) would be used to calculate the loss

In [17]:
#now let's get to the juict stuff and build our NN architecture

In [18]:
#we start with a bigram model
#bigram is when the model take 2(bi) words and trains on them to figure the realtion between them.
#ex 'Machine Learning' - the importance of 'Learning' is computed by looking at one word back i.e 'Machine'

In [19]:
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(22)

class BigramModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    #this is a lookup or hashmap table of vocab_size x vocab_size, where each token directly reads off the logits (scores) for next token from this table

  def forward(self, idx, targets=None):

    #idx and targets here are the (B,T) tensors of integers we split before.

    logits = self.token_embedding_table(idx) # (B,T,C) pytorch converts to torch of shape (batch, target, channels) here channels is vocab_size

    if targets is None:
      loss = None
    else:
      #we need to reshape our logits as pytorch expects shape (B*T, C) for input for cross entropy, so let's modify that
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  #now let's define our generate function which will predict the next token

  def generate(self, idx, max_new_tokens):
    #idx is (B,T) array of indices in the current context
    for _ in range(max_new_tokens):
      logits, loss = self(idx) #get the predictions
      #we want to focus only on last timestamp to predict next token
      logits = logits[:, -1, :] #becomes (B,C)
      #apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B,C)
      #sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
      #append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B,T+1)
    return idx

In [20]:
#let's test this out

m = BigramModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape) #shape of logits (B,T,C)
print(loss)

torch.Size([36, 65])
tensor(4.8492, grad_fn=<NllLossBackward0>)


In [21]:
idx = torch.zeros((1,1), dtype=torch.long)
generated_token = m.generate(idx, max_new_tokens=100)
print(decoder(generated_token[0].tolist()))

#before training, we get some garbage as you can see


flA-uPX
tZOF3LP&XTdoIVYgz,chayxaUmDOEnafCAZrnWIRDcbK3K.DFNQYHJ&rnHOG&It!?DcGTmCGBW,bfFeFYyis3Wk-jCuK


In [22]:
# so now it is training time. yum.

#create a pytorch optimizer. we will use AdamW. You can also use SGD

optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [23]:
#train loop

batch_size = 32
for steps in range (10000):
  #let us sample a batch of the data
  xb,yb = fetch_batch('train')

  #we now evaluate the loss
  logits, loss = m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.4050614833831787


In [27]:
print(decoder(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

#not great but we getting some sentence structure! so decent progress


Foul, wse mbikefota y n!
I anclivelle ar:
HNCAwipre soseatilibl:
Whe birst penser un wo ad he d, ces imivethourthay, s c me tooraknatt.
FFisthoryon I ald vellllt t mu t wetee, ithot ucowhrwnkld cuchy fr thep.

Fors be, waler g ghe ol anl mowe feve, ay ven s
SBy th oelean flk, weolD:

Thoome t buspomy thens tathe tol yay t byooofupt
Hin wacunske mothowhiserup, f he bl que uPr ccheawead thar athimigo pon arfonnth blond l ild ERERMPam a at p! aly RD:
ISOnce
Thean yourerresq:



Whantoouneep mirdlon
